In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
#常量
a = tf.constant(120, name="a")
b = tf.constant(130, name="b")
c = tf.constant(140, name="c")
#变量
v = tf.Variable(0, name="v")

#计算
calc_op = a + b + c
assign_op = tf.assign(v, calc_op)
sess = tf.Session()
sess.run(assign_op)

print(sess.run(v))

390


In [4]:
#预设值
a = tf.placeholder(tf.int32, [3]) #声明 长度为3的数组
b = tf.constant(2)
x2_op = a * b

sess = tf.Session()

r1 = sess.run(x2_op, feed_dict={ a:[1, 2, 3] })
print(r1)
r2 = sess.run(x2_op, feed_dict={ a:[10, 20, 30] })
print(r2)

[2 4 6]
[20 40 60]


In [6]:
a = tf.placeholder(tf.int32, [None]) #声明 长度为任意
b = tf.constant(2)
x2_op = a * b

sess = tf.Session()

r1 = sess.run(x2_op, feed_dict={ a:[1, 2, 3, 4, 5, 6] })
print(r1)
r2 = sess.run(x2_op, feed_dict={ a:[10, 20, 30] })
print(r2)

[ 2  4  6  8 10 12]
[20 40 60]


In [20]:
#深度学习挑战
#CSV 身高,体重,标签
csv = pd.read_csv("bmi.csv")
#数据整理
csv["height"] = csv["height"] / 200
csv["weight"] = csv["weight"] / 100
#标签的三种表现形式
bclass = {"thin": [1,0,0], "normal": [0,1,0], "fat":[0,0,1]}
csv["label_pat"] = csv["label"].apply(lambda x: np.array(bclass[x]))
#print(csv["label_pat"])

#测试数据
test_csv = csv[15000:20000]
test_pat = test_csv[["weight", "height"]]
test_ans = list(test_csv["label_pat"])
#print(test_pat)
#print(test_ans)

#准备训练数据
x = tf.placeholder(tf.float32, [None, 2]) #身体,体重数据
y_ = tf.placeholder(tf.float32, [None, 3]) #标签
#变量
W = tf.Variable(tf.zeros([2, 3])) #权重
b = tf.Variable(tf.zeros([3])) #偏向
#回归函数
y = tf.nn.softmax(tf.matmul(x, W) + b)

#模型训练
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
#反向计算
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(cross_entropy)

#正确率计算
#检测标签 argmax 获取其中的最大值的索引 及 返回的是模型对于任一输入x预测到的标签值 
predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
#reduce_mean跨维度计算平均值  cast [True，False，True，True] => [1, 0 ,1, 1]  ==> 0.75
#tf.equal 来检测我们的预测是否真实标签匹配(索引位置一样表示匹配)
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

#开始执行
sess = tf.Session()
sess.run(tf.global_variables_initializer()) #变量初始化
#训练
for step in range(3500):
    i = (step * 100) % 14000
    rows = csv[1 + i : 1 + i + 100]
    x_pat = rows[["weight", "height"]]
    y_ans = list(rows["label_pat"])
    fd = {x: x_pat, y_: y_ans}
    sess.run(train, feed_dict=fd)
    if step % 500 == 0:
        cre = sess.run(cross_entropy, feed_dict=fd) #预测
        acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans}) #计算
        print("step=", step, "cre=", cre, "acc=", acc)

#最终正确率
acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
print("正确率=", acc)

step= 0 cre= 108.663 acc= 0.3242
step= 500 cre= 57.5887 acc= 0.8904
step= 1000 cre= 45.0209 acc= 0.898
step= 1500 cre= 41.6543 acc= 0.9566
step= 2000 cre= 34.664 acc= 0.943
step= 2500 cre= 34.287 acc= 0.9674
step= 3000 cre= 26.8808 acc= 0.9726
正确率= 0.9712
